In [4]:
import faiss
import numpy as np


def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')


xb = read_fvecs('./dataset/sift/sift_base.fvecs')
xq = read_fvecs('./dataset/sift/sift_query.fvecs')[0].reshape(1, -1)
xq_full = read_fvecs('./dataset/sift/sift_query.fvecs')

In [7]:
d = 128 # dimensions
M = 32 # number of neighbours

index = faiss.IndexHNSWFlat(d, M)

In [12]:
index.hnsw.max_level

-1

In [15]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

In [16]:
index.add(xb)


In [17]:
index.hnsw.max_level


4

In [20]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1])

In [23]:
index.hnsw.entry_point

118295

In [24]:
def set_default_probas(M: int, m_L: float):
    nn = 0  # set nearest neighbours count
    cum_nneighbour_per_level = []
    level = 0  # we start at level 0
    assign_probas = []
    while True:
        # calculate probability for current level
        proba = np.exp(-level / m_L) * (1 - np.exp(-1 / m_L))
        # once we reach low prob threshold, we've created enough levels
        if proba < 1e-9: break
        assign_probas.append(proba)
        # neighbours is == M on every level except level 0 where == M*2
        nn += M * 2 if level == 0 else M
        cum_nneighbour_per_level.append(nn)
        level += 1
    return assign_probas, cum_nneighbour_per_level


In [27]:
assign_probas, cum_nneighbour_per_level = set_default_probas(M, 1 / np.log(M))
assign_probas, cum_nneighbour_per_level

([0.96875,
  0.030273437499999986,
  0.0009460449218749991,
  2.956390380859371e-05,
  9.23871994018553e-07,
  2.887099981307982e-08],
 [64, 96, 128, 160, 192, 224])

In [28]:
def random_level(assign_probas: list, rng):
    # get random float from 'r'andom 'n'umber 'g'enerator
    f = rng.uniform()
    for level in range(len(assign_probas)):
        # if the random float is less than level probability...
        if f < assign_probas[level]:
            # ... we assert at this level
            return level
        # otherwise subtract level probability and try again
        f -= assign_probas[level]
    # below happens with very low probability
    return len(assign_probas) - 1


In [30]:
chosen_levels = []
rng = np.random.default_rng(12345)
for _ in range(1_000_000):
    chosen_levels.append(random_level(assign_probas, rng))

In [32]:
np.bincount(chosen_levels)
# результат близок тому, который мы получили в hnsw индексе

array([968821,  30170,    985,     23,      1])

In [57]:
index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efConstruction = 32
index.add(xb) # build the index
index.hnsw.efSearch = 32
# and now we can search
index.search(xq_full[:1000], k=1)

(array([[ 54229.],
        [ 51187.],
        [ 30792.],
        [ 30510.],
        [ 47367.],
        [ 64458.],
        [ 50323.],
        [ 23986.],
        [ 45651.],
        [ 44167.],
        [ 54136.],
        [ 29640.],
        [ 39235.],
        [ 38222.],
        [ 56652.],
        [ 46372.],
        [ 34458.],
        [ 51553.],
        [ 50661.],
        [ 38574.],
        [ 20634.],
        [  6536.],
        [ 26959.],
        [ 42395.],
        [ 23786.],
        [ 48245.],
        [ 29865.],
        [ 30557.],
        [ 36383.],
        [ 15720.],
        [ 41257.],
        [ 11923.],
        [ 35180.],
        [ 48734.],
        [ 41196.],
        [ 14635.],
        [ 12423.],
        [ 27841.],
        [ 24448.],
        [ 21966.],
        [ 23582.],
        [ 32682.],
        [ 40497.],
        [ 27337.],
        [ 31957.],
        [ 29434.],
        [  9926.],
        [ 79168.],
        [ 24262.],
        [ 16585.],
        [ 32812.],
        [ 17112.],
        [ 22